In [148]:
import pandas as pd
import pickle

base_csv = '../data/pub_868.csv'
binary_feat = '../data/save_feats/detect_if_new_whale_bbox_xception_299_feat.pkl'
HIGH_THRES = 0.93
LOW_THRES = 0.07


df = pd.read_csv(base_csv)
df.head()

,Image,Id
0,00028a005.jpg,w_dec7ffd new_whale w_ab44ae4 w_f009245 w_2a0e...
1,000dcf7d8.jpg,w_4132bb8 new_whale w_3de579a w_9d29561 w_9119...
2,000e7c7df.jpg,w_94c3bf0 new_whale w_3de579a w_fc6dd24 w_d7ae...
3,0019c34f4.jpg,w_a335fc2 w_c99807e new_whale w_59052ad w_0135...
4,001a4d292.jpg,new_whale w_20eeb29 w_5b40032 w_ce66f9a w_82e7...


In [149]:
print(df.values[0])

['00028a005.jpg'
 'w_dec7ffd new_whale w_ab44ae4 w_f009245 w_2a0e817 w_700ebb4']


In [150]:
def keep5(s):
    return ' '.join(s.split(' ')[:5])

df.Id = df.Id.apply(keep5)
print(df.values[0])

['00028a005.jpg' 'w_dec7ffd new_whale w_ab44ae4 w_f009245 w_2a0e817']


In [151]:
with open(binary_feat,'rb') as fin:
    binary_f = pickle.load(fin)
print(binary_f.keys())
print(binary_f['pred']['00028a005.jpg'])

dict_keys(['pred', 'label_list'])
[0.20772535 0.7922747 ]


In [152]:
for k in binary_f['pred']:
    if binary_f['pred'][k][0]<0.07:
        print(k, binary_f['pred'][k])

1850cbe9f.jpg [0.06571262 0.9342874 ]
367b03954.jpg [0.06901273 0.93098724]
c881bb62d.jpg [0.06816036 0.9318396 ]
ba250a21e.jpg [0.06597461 0.9340254 ]
704a4bf34.jpg [0.06544501 0.93455493]
01d9c00d4.jpg [0.0499453  0.95005465]
f7663e305.jpg [0.06517369 0.93482625]
0cbfdf671.jpg [0.06868038 0.93131965]
195f7ca52.jpg [0.05358236 0.9464177 ]
e99148635.jpg [0.06514563 0.9348544 ]
106d84e98.jpg [0.06006038 0.9399396 ]
3180e1cd5.jpg [0.0537825 0.9462174]


In [153]:
NW = 'new_whale'

def update_label_str(label_str,p):
    tmp_list = label_str.split(' ')
    if p >= HIGH_THRES:
        if NW in label_str:
            idx = tmp_list.index(NW)
            if idx != 0:
                tmp_list[idx],tmp_list[idx-1] = tmp_list[idx-1],tmp_list[idx]
                return ' '.join(tmp_list)
        else:
            tmp_list = [NW] + tmp_list
            return ' '.join(tmp_list[:5])
        
    if p <= LOW_THRES:
        if NW in label_str:
            idx = tmp_list.index(NW)
            if idx != 4:
                tmp_list[idx],tmp_list[idx+1] = tmp_list[idx+1],tmp_list[idx]
                return ' '.join(tmp_list)
        else:
            pass
    
    return label_str

h_cnt = 0
l_cnt = 0
new_Id = []
for img_k, old_label in df.values:
    tmp_prob = binary_f['pred'][img_k]
    new_label_s = update_label_str(old_label,tmp_prob[0])
    new_Id.append(new_label_s)
    if new_label_s != old_label:
        print(old_label)
        print(new_label_s)
        print(tmp_prob)
        print('------')
        if new_label_s.startswith(NW):
            h_cnt += 1
        else:
            l_cnt += 1

w_6cda039 new_whale w_2b4b82e w_bd26110 w_ae32f8b
w_6cda039 w_2b4b82e new_whale w_bd26110 w_ae32f8b
[0.0499453  0.95005465]
------
w_fec331a new_whale w_3879fa0 w_2f1488c w_17b0d3a
w_fec331a w_3879fa0 new_whale w_2f1488c w_17b0d3a
[0.06868038 0.93131965]
------
new_whale w_4b5ade3 w_733f661 w_a94396f w_17b0d3a
w_4b5ade3 new_whale w_733f661 w_a94396f w_17b0d3a
[0.06006038 0.9399396 ]
------
w_3bfa4f8 new_whale w_3879fa0 w_f8bb266 w_d7aef56
new_whale w_3bfa4f8 w_3879fa0 w_f8bb266 w_d7aef56
[0.96110094 0.03889912]
------
new_whale w_2284d86 w_6b1d160 w_f13f7b3 w_d7998d1
w_2284d86 new_whale w_6b1d160 w_f13f7b3 w_d7998d1
[0.06571262 0.9342874 ]
------
w_263fcb0 w_28c1ced new_whale w_2284d86 w_a88f2c1
w_263fcb0 w_28c1ced w_2284d86 new_whale w_a88f2c1
[0.05358236 0.9464177 ]
------
w_e0f635c w_4df0da7 new_whale w_cf9055c w_281a8cd
w_e0f635c new_whale w_4df0da7 w_cf9055c w_281a8cd
[0.9530889  0.04691115]
------
w_c815410 new_whale w_60cf87c w_4b5ade3 w_7c9f76d
w_c815410 w_60cf87c new_whale w_4

In [154]:
print(h_cnt,l_cnt)
df.Id = new_Id
df.to_csv('../submissions/binary_up.csv',index=False)
df.head()

9 14


,Image,Id
0,00028a005.jpg,w_dec7ffd new_whale w_ab44ae4 w_f009245 w_2a0e817
1,000dcf7d8.jpg,w_4132bb8 new_whale w_3de579a w_9d29561 w_9119b08
2,000e7c7df.jpg,w_94c3bf0 new_whale w_3de579a w_fc6dd24 w_d7aef56
3,0019c34f4.jpg,w_a335fc2 w_c99807e new_whale w_59052ad w_0135f5f
4,001a4d292.jpg,new_whale w_20eeb29 w_5b40032 w_ce66f9a w_82e7319
